In [99]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aith-dl-competition-tabular-data/sample_submission.csv
/kaggle/input/aith-dl-competition-tabular-data/train.csv
/kaggle/input/aith-dl-competition-tabular-data/test.csv


In [100]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

Фиксация

In [101]:
PATH_TRAIN = '/kaggle/input/aith-dl-competition-tabular-data/train.csv'

In [102]:
PATH_TEST = '/kaggle/input/aith-dl-competition-tabular-data/test.csv'

In [103]:
data_train = pd.read_csv(PATH_TRAIN)

In [104]:
data_test = pd.read_csv(PATH_TEST)

Чтение данных и информация о них

In [105]:
data_test.head(2)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,15000,25.0,165.0,65.0,85.0,1.2,1.2,1.0,1.0,128.0,...,92.0,41.0,132.0,15.0,1.0,1.1,34.0,23.0,14.0,0.0
1,15001,45.0,165.0,60.0,74.0,1.5,1.0,1.0,1.0,104.0,...,124.0,54.0,129.0,11.3,1.0,0.7,20.0,17.0,11.0,0.0


In [106]:
data_train.head(2)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,35.0,175.0,75.0,86.5,1.2,1.2,1.0,1.0,127.0,...,58.0,108.0,15.6,1.0,0.9,17.0,14.0,21.0,0.0,0.0
1,1,45.0,155.0,60.0,82.0,1.2,1.0,1.0,1.0,129.0,...,50.0,110.0,14.0,1.0,0.7,22.0,18.0,14.0,0.0,0.0


In [107]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   15000 non-null  int64  
 1   age                  15000 non-null  float64
 2   height(cm)           15000 non-null  float64
 3   weight(kg)           15000 non-null  float64
 4   waist(cm)            15000 non-null  float64
 5   eyesight(left)       15000 non-null  float64
 6   eyesight(right)      15000 non-null  float64
 7   hearing(left)        15000 non-null  float64
 8   hearing(right)       15000 non-null  float64
 9   systolic             15000 non-null  float64
 10  relaxation           15000 non-null  float64
 11  fasting blood sugar  15000 non-null  float64
 12  Cholesterol          15000 non-null  float64
 13  triglyceride         15000 non-null  float64
 14  HDL                  15000 non-null  float64
 15  LDL                  15000 non-null 

In [108]:
data_train.shape

(15000, 24)

Подготовка данных

In [109]:
X = data_train.drop(['smoking', 'id'], axis=1)
y = data_train['smoking']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle = True, stratify = y, random_state=42)

#X_val = X_train.copy()

Стандартизация данных

In [110]:
scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_val_scaled = scaler.transform(X_val)

X_test = data_test.drop(['id'],axis=1)

X_test_scaled = scaler.transform(X_test)

(X_train_scaled.shape, X_val_scaled.shape, X_test_scaled.shape)

((12000, 22), (3000, 22), (10000, 22))

Преобразование в тензоры

In [111]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

Создание DataLoader

In [112]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=22, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=22, shuffle=False)

Определение модели

In [113]:
class SmoClassifier(nn.Module):
    def __init__(self, input_dim):
        super(SmoClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.3)
        self.fc4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout3(x)
        x = self.sigmoid(self.fc4(x))
        return x

In [114]:
input_dim = X_train.shape[1]
model = SmoClassifier(input_dim)

In [115]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Обучение модели

In [116]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    y_true_val = []
    y_scores_val = []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            y_true_val.extend(batch_y.numpy())
            y_scores_val.extend(outputs.numpy())
    
    y_true_val = np.array(y_true_val).flatten()
    y_scores_val = np.array(y_scores_val).flatten()
    roc_auc = roc_auc_score(y_true_val, y_scores_val)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation ROC-AUC: {roc_auc:.4f}')

Epoch [1/50], Loss: 0.2890, Validation ROC-AUC: 0.8772
Epoch [2/50], Loss: 0.3995, Validation ROC-AUC: 0.8798
Epoch [3/50], Loss: 0.4518, Validation ROC-AUC: 0.8820
Epoch [4/50], Loss: 0.8813, Validation ROC-AUC: 0.8823
Epoch [5/50], Loss: 0.3191, Validation ROC-AUC: 0.8843
Epoch [6/50], Loss: 0.4300, Validation ROC-AUC: 0.8832
Epoch [7/50], Loss: 0.2105, Validation ROC-AUC: 0.8843
Epoch [8/50], Loss: 0.3339, Validation ROC-AUC: 0.8854
Epoch [9/50], Loss: 0.3625, Validation ROC-AUC: 0.8840
Epoch [10/50], Loss: 0.4964, Validation ROC-AUC: 0.8837
Epoch [11/50], Loss: 0.3854, Validation ROC-AUC: 0.8824
Epoch [12/50], Loss: 0.4285, Validation ROC-AUC: 0.8841
Epoch [13/50], Loss: 0.2429, Validation ROC-AUC: 0.8841
Epoch [14/50], Loss: 0.6483, Validation ROC-AUC: 0.8865
Epoch [15/50], Loss: 0.2031, Validation ROC-AUC: 0.8849
Epoch [16/50], Loss: 0.4047, Validation ROC-AUC: 0.8858
Epoch [17/50], Loss: 0.2203, Validation ROC-AUC: 0.8874
Epoch [18/50], Loss: 0.2921, Validation ROC-AUC: 0.8871
E

Обучение модели на тестовых данных

In [121]:
X_test = torch.tensor(X_test_scaled, dtype=torch.float32)

In [122]:
test_dataset = TensorDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [123]:
model.eval()
y_scores_test = []

with torch.no_grad():
    for batch_X in test_loader:
        batch_X = batch_X[0]
        outputs = model(batch_X)
        y_scores_test.extend(outputs.numpy())

y_scores_test = np.round(np.array(y_scores_test).flatten(), 1)

Сохранение файла

In [125]:
submission_df = pd.DataFrame({
    'id': data_test['id'].values,
    'smoking': y_scores_test
})

submission_df.to_csv('sumple_submission.csv', index=False)
print("Файл sumple_submission.csv сохранен.")

Файл sumple_submission.csv сохранен.
